In [1]:
data_var = '2023-12-07'

In [2]:
import pandas as pd; pd.set_option('display.max_columns', None)
from datetime import date, timedelta
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [3]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [4]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [5]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [6]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [7]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
2328,2023-12-07,Brasil Nbb,18:00,São Paulo,Flamengo,2.65,1.44,156.5,1.87,1.79,3.5,2.08,1.67,vVZ8zwvg,0.377358,0.694444,0.534759,0.558659,0.071803,0.2,0.0,NaN,NaN,114.900,28.330366,0.246565,114.670,30.038025,0.261952,116.840,0.000,84.24,114.73,0.0,0.0,0.0,0.0,0.418386,0.030912,0.154621,-1.13,-0.226,-7.300885,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
2329,2023-12-07,Brasil Nbb,20:00,Franca,Brasília,1.04,9.85,164.5,1.85,1.85,-24.5,2.09,1.66,Y5lCZdga,0.961538,0.101523,0.540541,0.540541,0.063061,0.6,0.0,NaN,NaN,121.436,31.448645,0.258973,268.784,205.843653,0.765833,113.828,0.000,96.12,125.61,0.0,0.0,0.0,0.0,1.144097,0.000000,0.162163,-0.11,-0.022,-1.818182,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
2330,2023-12-07,Brasil Nbb,20:00,Pinheiros,Mogi,1.25,3.69,158.5,1.87,1.79,-9.5,1.98,1.74,CdmGYG85,0.800000,0.271003,0.534759,0.558659,0.071003,0.0,0.4,NaN,NaN,247.712,103.963976,0.419697,236.900,144.734306,0.610951,0.000,191.958,174.00,264.88,0.0,0.0,0.0,0.0,0.698518,0.030912,0.091240,0.00,0.000,inf,0.0,0.0,0.0,-1.56,-0.312,-8.621795,0.0,0.0,0.0
2331,2023-12-07,Eua Torneio Nba In,19:00,Milwaukee Bucks,Indiana Pacers,1.54,2.74,253.5,1.83,2.12,-5.5,2.04,1.78,8ruZkWKu,0.649351,0.364964,0.546448,0.471698,0.014314,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,223.38,0.00,0.0,0.0,0.0,0.0,0.396508,0.103828,0.096255,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
2332,2023-12-07,Eua Torneio Nba In,23:00,Los Angeles Lakers,New Orleans Pelicans,1.71,1.96,228.5,1.83,1.83,-3.5,2.06,1.65,IgMKQzgK,0.584795,0.510204,0.546448,0.546448,0.094999,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,195.04,297.18,0.0,0.0,0.0,0.0,0.096336,0.000000,0.156288,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2390,2023-12-07,Rússia Liga Vtb United,12:30,Minsk,Astana,2.94,1.41,156.5,1.87,1.92,4.5,2.07,1.72,W0UHHDn1,0.340136,0.709220,0.534759,0.520833,0.049356,0.0,0.0,NaN,NaN,518.118,235.370092,0.454279,295.796,151.821066,0.513263,0.000,0.000,851.40,447.12,1.0,0.0,0.0,0.0,0.497413,0.018657,0.130600,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
2391,2023-12-07,Rússia Liga Vtb United,13:30,Zenit Petersburg,Samara,1.17,5.15,157.5,1.82,1.94,-12.5,2.06,1.71,C891EpuG,0.854701,0.194175,0.549451,0.515464,0.048876,0.0,0.0,NaN,NaN,98.090,23.260148,0.237131,148.028,48.001916,0.324276,0.000,0.000,72.76,195.16,0.0,0.0,0.0,0.0,0.890597,0.045134,0.131293,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
2392,2023-12-07,Rússia Liga Vtb United,14:00,Runa Basket Moscow,Uralmash Ekaterinburg,1.60,2.36,163.5,1.82,1.94,-4.5,2.03,1.75,IoAcFQP9,0.625000,0.423729,0.549451,0.515464,0.048729,0.0,0.0,NaN,NaN,226.680,89.429321,0.394518,158.304,46.085424,0.291120,0.000,0.000,290.70,206.10,0.0,0.0,0.0,0.0,0.271415,0.045134,0.104757,0.00,0.000,inf,0.0,0.0,0.0,0.00,0.000,inf,0.0,0.0,0.0
2393,2023-12-07,Brasil Nbb,20:30,Paulistano,Botafogo,1.16,4.75,148.5,1.80,1.86,-12.5,2.09,1.66,0vy5yJOn,0.862069,0.210526,0.555556,0.537634,0.072595,0.8,0.8,NaN,NaN,128.458,41.889587,0.326096,279.376,161.198490,0.576995,140.402,447.630,180.00,526.68,0.0,0.0,0.0,0.0,0.859057,0.023184,0.162163,-0.17,-0.034,-4.705882,0.0,0.0,0.0,-5.00,-1.000,-3.750000,0.0,0.0,0.0


## Apostas do dia

### Over v0 (LDA)

In [8]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
3,19:00,Eua Torneio Nba In,Milwaukee Bucks,Indiana Pacers,253.5,1.83,1.0,Over
